In [29]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.metrics import classification_report
import pickle

In [30]:
X = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/X_train_agg.pkl', compression='gzip') 
y = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/y_train_agg.pkl', compression='gzip')             

In [31]:
xgb_classifier = xgb.XGBClassifier() #create XGB Classifier model with default parameters

In [32]:
xgb_classifier = xgb_classifier.fit(X,y) #fit model with aggregate training data

In [33]:
predict = xgb_classifier.predict(X) 

In [34]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [35]:
print(f'M = {amex_metric(pd.DataFrame(y), pd.DataFrame(predict, index=y.index, columns=["prediction"]))}')

M = 0.6600017053943578


In [36]:
print(classification_report(y, predict))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95    340085
           1       0.85      0.85      0.85    118828

    accuracy                           0.92    458913
   macro avg       0.90      0.90      0.90    458913
weighted avg       0.92      0.92      0.92    458913



In [38]:
pickle.dump(xgb_classifier, open('xgb_classifier_agg_model.sav', 'wb'))